In [1]:
import glob
import json
import sys

import numpy as np

sys.path.append('contrastive')

In [2]:
seeds = [7, 11, 13]
num_seeds = len(seeds)

## Table 1: Comparing performances

In [3]:
score_types = ['top', 'avg']

algorithms = [
    'sup',
    'arora',
    'stochastic',
    'deterministic',
    'pac-bayes'
]

In [4]:
models = ['cnn', 'mlp']

for model in models:
    table_1_data = {}
    for algorithm in algorithms:
        results = {'top1': [], 'top5': [], 'mu5-top1': [], 'mu5-top5': [], 'avg2': [], 'mu5-avg2': []}    
        for score_type in score_types:
            for seed in seeds:
                fname = 'results/{}/{}-{}-{}.json'.format(model, algorithm, score_type, seed)
                data = json.load(open(fname))
                for k, v in data.items():
                    results[k].append(v)
        table_1_data[algorithm] = results


    avg2_row = '& \\avgtwo  & '
    top1_row = '& \\topone  & '
    top5_row = '& \\topfive & '

    score_format = '${:.1f}$'.format
    for k, data in table_1_data.items():

        top1_row += score_format(np.array(data['top1']).mean())
        top1_row += ' & '
        top1_row += score_format(np.array(data['mu5-top1']).mean())
        top1_row += ' && '

        top5_row += score_format(np.array(data['top5']).mean())
        top5_row += ' & '
        top5_row += score_format(np.array(data['mu5-top5']).mean())
        top5_row += ' && '

        avg2_row += score_format(np.array(data['avg2']).mean())
        avg2_row += ' & '
        avg2_row += score_format(np.array(data['mu5-avg2']).mean())
        avg2_row += ' && '
    print('\\\\ \n'.join([avg2_row[:-3], top1_row[:-3], top5_row[:-3]]) + '\\\\')
    if model == 'cnn':
        print('\\midrule\n\\multicolumn{2}{c}{\\auslan} \\\\')


& \avgtwo  & $91.4$ & $87.5$ && $89.4$ & $85.6$ && $87.7$ & $83.9$ && $90.0$ & $87.2$ && $75.4$ & $70.8$ \\ 
& \topone  & $25.3$ & $16.8$ && $22.5$ & $15.6$ && $17.3$ & $12.7$ && $21.4$ & $16.0$ && $6.9$ & $5.4$ \\ 
& \topfive & $57.8$ & $46.0$ && $52.9$ & $42.6$ && $46.9$ & $38.3$ && $54.0$ & $45.2$ && $23.4$ & $19.4$ \\
\midrule
\multicolumn{2}{c}{\auslan} \\
& \avgtwo  & $80.2$ & $75.1$ && $85.6$ & $83.3$ && $85.3$ & $82.7$ && $85.3$ & $82.9$ && $82.6$ & $79.1$ \\ 
& \topone  & $12.0$ & $7.1$ && $38.0$ & $24.9$ && $36.1$ & $23.7$ && $37.1$ & $24.7$ && $23.2$ & $14.8$ \\ 
& \topfive & $35.7$ & $24.1$ && $56.7$ & $48.2$ && $56.2$ & $47.7$ && $56.5$ & $49.1$ && $50.6$ & $38.4$ \\


## Table 2: Report generalisation error bounds

In [5]:
def get_best_model_name(model_name_dir, criterion_key_in_json='lowest_val_loss'):

    if model_name_dir[-1] == '/':
        model_name_dir = model_name_dir[:-1]
    fnames = glob.glob('{}/*.json'.format(model_name_dir))

    lowest_val_loss = np.finfo(np.float(0.)).max

    best_model_fname = ''
    for fname in fnames:
        result = json.load(open(fname))
        if criterion_key_in_json not in result:
            print('{} field is not found in {}'.format(criterion_key_in_json, fname))
            continue

        if result[criterion_key_in_json] < lowest_val_loss:
            lowest_val_loss = result[criterion_key_in_json]
            best_model_fname = fname

    return best_model_fname.replace('json', 'pt')


In [6]:
def get_optimal(terms) -> tuple:
    lambda_array = np.arange(10, 10**7, 1)
    bounds = catoni_bound(lambda_array, terms)
    optimized_pb_bound = min(bounds)
    optimal_lambda = lambda_array[np.argmin(bounds)]
    return optimal_lambda, optimized_pb_bound


def catoni_bound(l, terms) -> np.ndarray:
    M = terms['m']
    inner_exp = l / M  * terms['train-zero-one-loss'] + (terms['complexity'] + np.log(2 * np.sqrt(M))) / M
    return (1. - np.exp(-inner_exp)) / (1. - np.exp(- l / M))        


In [7]:
float_format = ' ${:.3f}$ &'.format
lambda_format = ' $10^{}$ &'.format
kl_format = ' ${:,d}$ &'.format
optimal_lambda_format = ' ${:,}$ &'.format
line_break = '\\\\'
models = ['cnn', 'mlp']

val_algorithms = [
    'stochastic',
    'deterministic',
]


In [8]:
# CNN and iid AUSLAN

results = []

for model in models:
    emp_risk_row = '& $\\Runhat(\\Qcal)$ &'
    test_risk_row = '& $\\Run(\\Qcal)$ &'
    emp_risk_f_row = '& $\\Runhat(\\fbf^*)$ &'
    test_risk_f_row = '& $\\Run(\\fbf^*)$ &'
    bound_row = '& Bound &'
    kl_row = '& $\\KL$ &'
    fixed_lambda_row = '& $\\lambda\\times m$ &'
    optimal_lambda_row = '& $\widehat{\lambda}\\times m$ &'
    
    # calculate PAC-Bayes bound and risks for best stochastic/deterministic models with respect to validation risk.
    for algorithm in val_algorithms:
        emp_risk = 0. 
        test_risk = 0.
        emp_risk_f = 0. 
        test_risk_f = 0.
        bound = 0.
        kl = 0.
        fixed_lambda = 0.
        sum_optimal_lambda = 0.
        for seed in seeds:
            fname = get_best_model_name(model_name_dir='weights/{}/{}/seed-{}'.format(model, algorithm, seed))
            data = json.load(open('./bounds/{}/pac-bayes-{}-{}.json'.format(model, seed, algorithm)))
            terms = data[fname]
            
            terms_f = json.load(open('./bounds/{}/pac-bayes-{}-{}-det.json'.format(model, seed, algorithm)))[fname]
            emp_risk_f += terms_f['train-zero-one-loss']
            test_risk_f += terms_f['test-zero-one-loss']
            del terms_f

            optimal_lambda, optimized_pb_bound = get_optimal(terms)
            
            print('\r', model, algorithm, seed, optimized_pb_bound, optimal_lambda, terms['lambda'], end='')

            emp_risk += terms['train-zero-one-loss']
            test_risk += terms['test-zero-one-loss']
            bound += optimized_pb_bound
            kl +=  terms['kl']
            fixed_lambda += terms['lambda']
            sum_optimal_lambda += optimal_lambda
            
        emp_risk_row += float_format(emp_risk / num_seeds)
        test_risk_row +=  float_format(test_risk / num_seeds)
        emp_risk_f_row += float_format(emp_risk_f / num_seeds)
        test_risk_f_row +=  float_format(test_risk_f / num_seeds)        
        bound_row += float_format(bound / num_seeds)
        kl_row += kl_format(int(kl / num_seeds))
        fixed_lambda_row +=  lambda_format(int(np.log10(fixed_lambda / num_seeds)))
        optimal_lambda_row += optimal_lambda_format(int(sum_optimal_lambda / num_seeds))
    
    # calculate PAC-Bayes bound and risks for best models with respect to the PAC-Bayes bound.
    emp_risk = 0.
    test_risk = 0.
    emp_risk_f = 0.
    test_risk_f = 0.    
    bound = 0.
    kl = 0.
    fixed_lambda = 0.
    sum_optimal_lambda = 0.    
    for seed in seeds:
        data = json.load(open('./bounds/{}/pac-bayes-{}.json'.format(model, seed)))
        data_f = json.load(open('./bounds/{}/pac-bayes-{}-det.json'.format(model, seed)))
        
        best_l = 0
        best_bound = np.finfo(np.float(0.)).max
        best_terms = {}
        best_terms_f = {}
        for k, terms in data.items():
            optimal_lambda, optimized_pb_bound = get_optimal(terms)
            if optimized_pb_bound < best_bound:
                best_l = optimal_lambda
                best_terms = terms
                best_bound = optimized_pb_bound
                best_terms_f = data_f[k]               

        print('\r', model, 'PAC-Bayes', seed, best_bound, best_l, best_terms['lambda'], end='')

        emp_risk += best_terms['train-zero-one-loss']
        test_risk += best_terms['test-zero-one-loss']
        emp_risk_f += best_terms_f['train-zero-one-loss']
        test_risk_f += best_terms_f['test-zero-one-loss']
        bound += best_bound
        kl +=  int(best_terms['kl'])
        fixed_lambda += best_terms['lambda']
        sum_optimal_lambda += best_l
        
    print('\r', end='')

    emp_risk_row += float_format(emp_risk / num_seeds) + line_break
    test_risk_row += float_format(test_risk / num_seeds ) + line_break
    emp_risk_f_row += float_format(emp_risk_f / num_seeds) + line_break
    test_risk_f_row += float_format(test_risk_f / num_seeds ) + line_break    
    bound_row += float_format(bound / num_seeds) + line_break
    kl_row += kl_format( int(kl / num_seeds))  + line_break
    fixed_lambda_row += lambda_format(int(np.log10(fixed_lambda / num_seeds))) + line_break
    optimal_lambda_row += optimal_lambda_format(int(sum_optimal_lambda / num_seeds)) + line_break

    results.append('\n'.join([emp_risk_f_row, test_risk_f_row, emp_risk_row, test_risk_row, bound_row, kl_row, fixed_lambda_row, optimal_lambda_row]))

    if model == 'cnn':
        results.append('\n\\midrule\n\\multicolumn{2}{l}{\\auslan} \\\\\n')


In [9]:
print(''.join(results).replace(',', '\\,').replace('&\\\\', '\\\\'))

& $\Runhat(\fbf^*)$ & $0.146$ & $0.131$ & $0.308$ \\
& $\Run(\fbf^*)$ & $0.185$ & $0.167$ & $0.315$ \\
& $\Runhat(\Qcal)$ & $0.172$ & $0.170$ & $0.323$ \\
& $\Run(\Qcal)$ & $0.203$ & $0.197$ & $0.327$ \\
& Bound & $0.733$ & $0.718$ & $0.437$ \\
& $\KL$ & $32\,756$ & $30\,894$ & $1\,333$ \\
& $\lambda\times m$ & $10^5$ & $10^5$ & $10^4$ \\
& $\widehat{\lambda}\times m$ & $122\,781$ & $119\,687$ & $24\,295$ \\
\midrule
\multicolumn{2}{l}{\auslan} \\
& $\Runhat(\fbf^*)$ & $0.193$ & $0.190$ & $0.263$ \\
& $\Run(\fbf^*)$ & $0.182$ & $0.182$ & $0.216$ \\
& $\Runhat(\Qcal)$ & $0.199$ & $0.195$ & $0.267$ \\
& $\Run(\Qcal)$ & $0.186$ & $0.185$ & $0.220$ \\
& Bound & $0.419$ & $0.417$ & $0.361$ \\
& $\KL$ & $9\,769$ & $10\,018$ & $2\,054$ \\
& $\lambda\times m$ & $10^5$ & $10^5$ & $10^4$ \\
& $\widehat{\lambda}\times m$ & $95\,683$ & $97\,379$ & $45\,198$ \\
